### Problem analysis

In [1]:
import pandas as pd
import cProfile

In [2]:
df = pd.read_csv("../notes/protocol_default.csv", comment="#", delim_whitespace=True)
df

/var/folders/pb/8wrfvtz5593cjrgr_g89r70m0000gn/T/ipykernel_39734/2015479.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("../notes/protocol_default.csv", comment="#", delim_whitespace=True)


,Problem,Status,Usertime,Failure,Version,Preprocessingtime,Backward_subsumed,Factors_computed,Forward_subsumed,Initial_clauses,Processed_clauses,Resolvents_computed,Systemtime,Tautologies_deleted,Totaltime
0,AGT001+1.p,T,1.031,success,1.2,-,0,1,40,652,405,531,0.017,3,1.048
1,AGT001+2.p,T,1.997,success,1.2,-,0,1,129,1019,752,942,0.020,3,2.017
2,AGT002+1.p,T,0.887,success,1.2,-,0,1,40,652,264,400,0.015,3,0.902
3,AGT002+2.p,T,1.721,success,1.2,-,0,1,129,1019,610,809,0.020,3,1.741
4,AGT003+1.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16089,TOP047+4.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
16090,TOP048+1.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
16091,TOP048+2.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
16092,TOP048+3.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-


In [3]:
df = df.sort_values("Usertime", ascending=False)
df

,Problem,Status,Usertime,Failure,Version,Preprocessingtime,Backward_subsumed,Factors_computed,Forward_subsumed,Initial_clauses,Processed_clauses,Resolvents_computed,Systemtime,Tautologies_deleted,Totaltime
13131,SWV194+1.p,T,99.852,success,1.2,-,4,1,1960,357,2166,98975,0.371,2,100.223
5358,LAT261-1.p,T,99.759,success,1.2,-,12,5,5673,3038,3687,61459,0.138,72,99.897
11919,SWB011+1.p,T,99.539,success,1.2,-,25,198,4675,1281,3675,43267,0.120,58,99.659
5094,LAT005-2.p,T,98.176,success,1.2,-,55,0,4391,31,3966,9397,0.032,339,98.208
2021,FLD021-3.p,T,97.506,success,1.2,-,6,8,96,30,1970,277716,0.371,10,97.877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5722,LAT362+3.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5723,LAT362+4.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5724,LAT363+1.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5725,LAT363+2.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-


In [4]:
import pyres_fof
import cProfile
import re

opts = [
    ("-s", ""),
    ("-r", "20"),
]
args = [
    "EXAMPLES/PUZ002-1.p"
]
with cProfile.Profile() as pr:
    pyres_fof.main(from_notebook=True, notebook_opts=opts, notebook_args=args)
    df_stats = pd.DataFrame(
        pr.getstats(),
        columns=["func", "ncalls", "ccalls", "tottime", "cumtime", "callers"]
    )
df_stats["func"] = df_stats["func"].astype(str)
df_stats = df_stats[ df_stats["func"].str.contains("alternatingpath.py") ]

func_names = []
lines = []
for value in df_stats["func"]:
    print(value)
    func_name = re.search(r'(?<=code object )(\w+)', value)
    line = re.search(r'(?<=line )(\d+)', value)

    func_names.append(func_name.group() if func_name is not None else None)
    lines.append(int(line.group()) if line is not None else None)

df_stats["func_name"] = func_names
df_stats["line"] = lines
print(df_stats.columns)
df_stats = df_stats[["func_name", "line", "ncalls", "ccalls", "tottime", "cumtime", "callers"]]


df_stats.to_csv("profiling_runs.csv")
df_stats

% SZS status Unsatisfiable

% Initial clauses    : 12
% Processed clauses  : 22
% Factors computed   : 0
% Resolvents computed: 22
% Tautologies deleted: 0
% Forward subsumed   : 0
% Backward subsumed  : 0
% -------- CPU Time ---------
% User time          : 0.441 s
% System time        : 0.117 s
% Total time         : 0.558 s
<code object get_neighbours at 0x10a9eb830, file "/Users/jannisgehring/VSCode/coursework-pyres/PyRes/alternatingpath.py", line 87>
<code object construct_nodes at 0x10ab215f0, file "/Users/jannisgehring/VSCode/coursework-pyres/PyRes/alternatingpath.py", line 34>
<code object clauses_to_nodes at 0x10acdfaa0, file "/Users/jannisgehring/VSCode/coursework-pyres/PyRes/alternatingpath.py", line 76>
<code object __init__ at 0x10acf26b0, file "/Users/jannisgehring/VSCode/coursework-pyres/PyRes/alternatingpath.py", line 28>
<code object edge_neighb_of_subset at 0x10ad170f0, file "/Users/jannisgehring/VSCode/coursework-pyres/PyRes/alternatingpath.py", line 83>
<code object

,func_name,line,ncalls,ccalls,tottime,cumtime,callers
51,get_neighbours,87,39,0,0.001526,0.000203,"[(<method 'append' of 'list' objects>, 60, 0, ..."
84,construct_nodes,34,1,0,0.000035,0.000025,"[(<code object __init__ at 0x10ad3a790, file ""..."
91,clauses_to_nodes,76,1,0,0.000002,0.000001,"[(<code object get_all_nodes at 0x10ad31930, f..."
96,__init__,28,1,0,0.002360,0.000010,"[(<code object construct_nodes at 0x10ab215f0,..."
104,edge_neighb_of_subset,83,1638,0,0.001050,0.001050,None
108,nodes_to_clauses,68,1,0,0.000026,0.000011,"[(<code object __init__ at 0x10ab1b690, file ""..."
110,get_rel_neighbourhood,101,1,0,0.001580,0.000021,[(<code object clauses_to_nodes at 0x10acdfaa0...
129,get_all_nodes,65,1,0,0.000001,0.000001,None
130,None,79,44,0,0.000004,0.000004,None
141,__init__,9,44,0,0.000005,0.000005,None
